In [ ]:
%pip install -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.1 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [ ]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
    # other params...
)

In [ ]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to Hindi. Translate the user sentence.",
    ),
    ("human", "My name is Tanay."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='मेरा नाम तनय है।', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--6d5be157-09d1-48db-84fb-4ace716c4e96-0', usage_metadata={'input_tokens': 23, 'output_tokens': 103, 'total_tokens': 126, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 97}})

In [ ]:
print(ai_msg.content)

मेरा नाम तनय है।


## Chaining

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
    # other params...
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_languages}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm | StrOutputParser()
chain.invoke(
    {
        "input_language": "English",
        "output_languages": ["Hindi", "Mandarin"],
        "input": "My name is Tanay."
    }
)

'**Hindi:** मेरा नाम तनाय है। (Mera naam Tanay hai.)\n**Mandarin:** 我叫 Tanay。 (Wǒ jiào Tanay.)'

# **POC - INTERVIEW QUESTIONS GENERATOR FROM RESUME**

In [ ]:
!pip install google-generativeai pdfminer.six dateparser unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 50.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 12.0 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=3a08a28282e6c1aba3e2dec5dd7d838b2e6fd3eac7da0ad31e08987029679f6e
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa3

In [ ]:
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

resume = StringIO()
with open('/content/Tanay Mehendale_Resume_DE13.pdf', 'rb') as in_file:
    parser = PDFParser(in_file)
    doc = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, resume, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.create_pages(doc):
        interpreter.process_page(page)

print(resume.getvalue())
raw_resume_text = resume.getvalue()

Tanay Mehendale 
San Jose, CA (can relocate) | tanay.mehendale@tamu.edu | 979-344-3679 | LinkedIn/tanay-mehendale/ | Portfolio 

SUMMARY 
Data Engineer with 2+ years of hands-on experience in building ETL pipelines, automating data workflows, and 
bridging SAP systems with cloud platforms. Proficient in both traditional enterprise data (SAP, SQL Server) and 
modern tools like Spark, Airflow, and AWS. Currently learning Snowflake and GenAI to modernize data workflows. 

EXPERIENCE 

Data Engineer - LTIMindtree | Mumbai, India 
Jan 2021 – Jun 2023 
•  Developed 45+ SQL programs to deliver SAP reports and custom transactions, reducing data inconsistencies 

by 18% and improving day-to-day operations for 30+ stakeholders across 5 global business teams 

•  Delivered an ETL pipeline using PySpark and AWS Redshift, processing big data volumes of 750GB/month 

that reduced reporting latency by 87% and saved approx. 15 analyst hours per week 
• 
Implemented data quality checks with Python, red

In [ ]:
prompt = f"""
You are an interview preparation coach.

Here is a candidate's resume text:

<<<
{raw_resume_text}
>>>

Tasks:
1. Estimate the candidate's total years of professional experience based on date ranges in the Experience section.
   - Handle formats like "Apr 2025", "April 2025", "April '24", "2021–2023", or "Present/Current".
   - Return your estimate in years with 1 decimal precision.

2. For each bullet point under Experience, generate at least 1 possible interview question.
   For each question, use the format:

{{
Question #. Question

Why They'll Ask This:
Reason

How To Prepare:
Helpful Tips

Sample Answer (STAR):
Situation – Set the context from the resume or a reasonable story.
Task – What was your responsibility or challenge?
Action – What concrete steps did you take?
Result – What outcome did your work produce? Quantify if possible.
}}

- If the bullet point doesn’t contain enough detail to make a story, create the best possible realistic STAR-style story based on the candidate’s role, years of experience, company, and domain.
- Make the answers sound authentic, simple and not generic or AI fluff — specific but grounded in the resume text.

3. For each project, generate technical interview questions appropriate for the candidate's years of experience (calculated in step 1).

4. Finally, generate 5 common behavioral questions, with the same 4-part format.
   Sample answers should reference resume experiences when possible.

Rules:
- Do not invent employers or degrees not in the resume, but you MAY expand bullets into plausible STAR stories.
- If dates are ambiguous, make the best reasonable guess.
- Always return answers in the strict 4-part format shown.
- Keep answers concise (4–7 sentences max).
"""


In [ ]:
import google.generativeai as genai
from google.colab import userdata

genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

model = genai.GenerativeModel("gemini-2.5-flash")  # or "gemini-1.5-flash" for faster
response = model.generate_content(prompt)

print(response.text)


Here's a comprehensive interview preparation guide based on Tanay Mehendale's resume:

---

### **1. Total Years of Professional Experience Estimate**

*   **LTIMindtree:** Jan 2021 – Jun 2023 = 2 years 5 months (2.41 years)
*   **Texas A&M University (Data Analyst):** Apr 2024 – May 2025 = 1 year 1 month (1.08 years)

**Total Estimated Professional Experience:** 3.5 years

---

### **2. Interview Questions for Experience Bullet Points**

**Data Engineer - LTIMindtree | Mumbai, India**

{
Question 1. Can you describe a specific instance where you developed a complex SQL program for an SAP report? What was the challenge, and how did your program reduce inconsistencies by 18%?

Why They'll Ask This:
This question assesses your technical SQL skills, understanding of business impact, problem-solving abilities, and experience working with enterprise systems like SAP. They want to see how you translated a business need into a technical solution.

How To Prepare:
Recall a specific SAP reporti

In [ ]:
!pip install rich

In [ ]:
from rich.console import Console
from rich.markdown import Markdown

console = Console()
console.print(Markdown(response.text))


Here's a comprehensive interview preparation guide based on Tanay Mehendale's resume:                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                1. Total Years of Professional Experience Estimate                                 

 • LTIMindtree: Jan 2021 – Jun 2023 = 2 years 5 months (2.41 years)                                                
 • Texas A&M University (Data Analyst): Apr 2024 – May 2025 = 1 year 1 month (1.08 years)                          

Total Estimated Professional Experience: 3.5 years                                                                 

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                2. Interview Questions for Experience Bullet Points                                

Data Engineer - LTIMindtree | Mumbai, India                                                                        

{ Question 1. Can you describe a specific instance where you developed a complex SQL program for an SAP report?    
What was the challenge, and how did your program reduce inconsistencies by 18%?                                    

Why They'll Ask This: This question assesses your technical SQL skills, understanding of business impact,          
problem-solving abilities, and experience working with enterprise systems like SAP. They want to see how you       
translated a business need into a technical solution.                                                              

How To Prepare: Recall a specific SAP reporting requirement. Be ready to detail the SQL logic (e.g., complex joins,
subqueries, CTEs, window functions), how you identified and measured data inconsistencies, and the validation      
process you used.                                                                                                  

Sample Answer (STAR): Situation – A key business team relied on manual exports from SAP for their weekly sales     
forecast, leading to frequent data discrepancies due to inconsistent filtering and joins. This affected over 30    
stakeholders across multiple regions. Task – My task was to automate and standardize the sales reporting process by
developing a robust SQL program that would directly query SAP data, eliminating manual intervention and improving  
data accuracy for sales forecasting. Action – I worked closely with the sales team to understand their exact       
reporting logic and identified common pitfalls. I then developed a series of complex SQL programs utilizing        
specific SAP table joins and views to extract sales orders, delivery statuses, and customer information. I         
implemented validation checks within the SQL to flag potential inconsistencies and worked on data cleansing scripts
before final aggregation. Result – The new SQL programs automated the report generation, ensuring consistent data. 
Post-implementation, we measured an 18% reduction in data inconsistencies in the weekly sales reports,             
significantly improving the reliability of the sales forecast and saving the business team several hours each week.
}                                                                                                                  

{ Question 2. Tell me about the ETL pipeline you built with PySpark and AWS Redshift. What was the most challenging
aspect of handling 750GB/month, and how did you achieve such a significant reduction in reporting latency?         

Why They'll Ask This: This question probes your Big Data experience, specific technology expertise (PySpark,       
Redshift), ability to design and optimize ETL processes, and understanding of the impact of your work on business  
operations.                                                                                                        

How To Prepare: Discuss the source data char